In [5]:
import torch


In [6]:
inputs = torch.nn.Embedding(4,8)
inputs = inputs.weight.data
inputs

tensor([[ 0.0976,  0.0149, -1.1556,  0.3064, -0.4257, -0.8682, -1.1081,  0.4433],
        [-0.7731,  0.7909, -0.9703,  0.2230,  1.5777,  1.1463, -0.7104,  1.2075],
        [ 0.3974,  0.2502, -1.4841,  0.5807,  0.0462,  0.3138,  0.1769,  1.4311],
        [ 0.4488, -2.3937,  0.3993,  0.9518, -0.6984,  1.2754, -0.3575,  2.3780]])

In [7]:
#create weight matrices
d_in = 8
d_out = 6
W_q = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
W_k = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
W_v = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)

In [8]:

#choose input vector and transform it into our query vector using W_q

query = inputs[2] @W_q
query



tensor([1.0150, 0.2433, 0.3195, 1.4967, 0.2781, 1.8095])

In [9]:
#calcualte attention scores using keys generated by w_k
keys = inputs @ W_k
values = inputs @ W_v
keys,values

(tensor([[-0.7094, -1.7580, -2.4617, -1.6822, -1.3071, -1.1707],
         [ 1.7877,  2.0714,  1.4485,  1.2207,  1.5358,  0.2700],
         [ 2.0347,  1.5154,  0.4382,  1.6824,  0.7576, -0.0227],
         [ 1.3624,  2.5922,  0.2755,  1.5078,  0.8509,  0.5176]]),
 tensor([[-1.1350, -1.4415, -2.0172, -1.7343, -1.0092, -1.8528],
         [ 2.5206,  0.4336,  1.4964,  1.1368,  2.5036,  2.3060],
         [ 1.8500,  0.6478, -0.1336,  1.4324,  1.2739,  1.2857],
         [ 0.9185,  3.6437, -0.2990,  2.4324, -0.1226,  1.6265]]))

In [10]:
attention_scores = query @ keys.T
attention_scores

tensor([-6.9339,  5.5239,  5.2617,  5.5315])

In [11]:
attentions_weights = torch.softmax(attention_scores / keys.shape[-1] ** 0.5, dim = -1)
attentions_weights

tensor([0.0021, 0.3439, 0.3090, 0.3450])

In [12]:
attentions_weights.sum()

tensor(1.)

In [13]:
context_vector = attentions_weights @ values
context_vector

tensor([1.7530, 1.6032, 0.3659, 1.6690, 1.2102, 1.7475])

In [ ]:
class SimpleAttention (torch.nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        #weight matrices
        self.W_q = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
        self.W_k = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
        self.W_v = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
    
    def forward(self, x):
        #x is embedding vectors
        queries = x @ self.W_q
        keys = x @ self.W_k
        values = x @ self.W_v
        scores = x @ keys.T
        weights = torch.softmax(scores/ keys.shape[-1] ** 0.5, dim = 1)
        context = weights @ values
        return context_vector
        

In [15]:
simple = SimpleAttention(d_in = 8, d_out = 6,)

In [18]:
simple.W_v

Parameter containing:
tensor([[0.1700, 0.5894, 0.9822, 0.8190, 0.2871, 0.6782],
        [0.2091, 0.5122, 0.1661, 0.3575, 0.1665, 0.2640],
        [0.6082, 0.2137, 0.2877, 0.2563, 0.7847, 0.9044],
        [0.4106, 0.4540, 0.9895, 0.3134, 0.0701, 0.5051],
        [0.2920, 0.7795, 0.2051, 0.9461, 0.7950, 0.2462],
        [0.7986, 0.9322, 0.1874, 0.1071, 0.0165, 0.3209],
        [0.6732, 0.9297, 0.3402, 0.7900, 0.2539, 0.8203],
        [0.8293, 0.3794, 0.0842, 0.0032, 0.7406, 0.2507]])

In [20]:
context_vectors = simple(inputs)
context_vectors

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x8 and 6x4)

In [ ]:
#second version but its linear not rand

class SimpleAttention (torch.nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        #weight matrices

        self.W_q = nn.Linear(d_in, d_out, bias = False)
        self.W_k = nn.Linear(d_in, d_out, bias = False)  
        self.W_v = nn.Linear(d_in, d_out, bias = False)


        
    def forward(self, x):
        #x is embedding vectors
        queries = x @ self.W_q
        keys = x @ self.W_k
        values = x @ self.W_v
        scores = x @ keys.T
        weights = torch.softmax(scores/ keys.shape[-1] ** 0.5, dim = 1)
        context = weights @ values
        return context_vector